# France - Final Model

## Fetch Pre-requisites

In [1]:
from typing import Tuple

import itertools
import logging

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

In [2]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('prophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('fbprophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [3]:
country = 'france'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

## Data Preparation

In [4]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    df_covid_holidays = pandas.DataFrame()
    return df_mrd, df_covid_holidays

In [5]:
df_mrd, _ = make_dataset(df)

In [6]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [7]:
df_train, df_test = train_test_split(df_mrd, 12)

## Model Training and Testing

In [8]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min'):
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params, interval_width=0.95).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params, interval_width=0.95).fit(df)
    return optimal_model

In [9]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

In [10]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [11]:
df_predicted, rmse = test_model(df_test, model)

In [12]:
df_predicted[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]

,ds,yhat,yhat_upper,yhat_lower
0,2022-03-01,7.447797,7.878150,7.054356
1,2022-04-01,7.406294,7.776836,7.018766
2,2022-05-01,7.379924,7.779443,6.955099
3,2022-06-01,7.360605,7.830526,6.935245
4,2022-07-01,7.360297,7.836046,6.886344
5,2022-08-01,7.317266,7.807194,6.824332
6,2022-09-01,7.290787,7.885154,6.766877
7,2022-10-01,7.253303,7.901592,6.579447
8,2022-11-01,7.237611,8.076996,6.565069
9,2022-12-01,7.202758,8.036206,6.425145


In [13]:
rmse

0.09831839132623509

## Final Model

In [14]:
model = train_model(df_mrd, param_grid)

### Save Model

In [15]:
with open(f'{country}_final_model.json', 'w') as f:
    f.write(model_to_json(model))

## Final Results

In [16]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [17]:
df_future_prediction = model.predict(df_future)

In [18]:
df_future_prediction[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]

,ds,yhat,yhat_upper,yhat_lower
0,2023-03-01,6.969797,7.340096,6.599593
1,2023-04-01,6.918349,7.339634,6.505892
2,2023-05-01,6.905033,7.307189,6.508546
